In [2]:
import pandas as pd
from pathlib import Path

#SETTINGS
FILE = Path("datasets/AA_ATFM_Delay.xlsx")
SHEET = "Blad1"
AIRPORTS_20 = [
    "LTFM","EHAM","EGLL","LFPG","EDDF","LEMD","LEBL","EDDM","LIRF","EGKK",
    "LSZH","LGAV","EIDW","LOWW","LEPA","EKCH","LPPT","LTFJ","LTAI","ENGM"
]
YEARS = [2023, 2024]   # use [2023] for only one year

#setting the letter into what it is
delay_mapping = {
    "A": "Accident/Incident",
    "C": "ATC Capacity",
    "D": "De-icing",
    "E": "Equipment (non-ATC)",
    "G": "Aerodrome Capacity",
    "I": "Industrial Action (ATC)",
    "M": "Airspace Management",
    "N": "Industrial Action (non-ATC)",
    "O": "Other",
    "P": "Special Event",
    "R": "ATC Routeing",
    "S": "ATC Staffing",
    "T": "Equipment (ATC)",
    "V": "Environmental Issues",
    "W": "Weather",
    "NA": "Not specified"
}

#LOAD & FILTER
df = pd.read_excel(FILE, sheet_name=SHEET)
df = df[df["YEAR"].isin(YEARS) & df["APT_ICAO"].isin(AIRPORTS_20)].copy()

#what columns
delay_cols = sorted([
    c for c in df.columns
    if c.startswith("DLY_APT_ARR_") and c.endswith("_1") and c != "DLY_APT_ARR_1"
])

#FIND MOST-OCCURRING TYPE PER (airport, year, month)(idk chat did this)
occ = (df[delay_cols].fillna(0) > 0)
group_keys = ["APT_ICAO", "YEAR", "MONTH_NUM"]
counts = occ.groupby([df[k] for k in group_keys]).sum()

best_col = counts.idxmax(axis=1)
best_code = (best_col.str.replace("DLY_APT_ARR_", "", regex=False)
                      .str.replace("_1", "", regex=False))

#table making
long_df = (
    best_code.reset_index(name="delay_code")
    .assign(
        Month=lambda d: d["YEAR"].astype(str) + "-" + d["MONTH_NUM"].astype(str).str.zfill(2),
        delay_type=lambda d: d["delay_code"].map(delay_mapping)
    )
    .loc[:, ["Month", "APT_ICAO", "delay_type"]]
    .sort_values(["APT_ICAO", "Month"])
    .set_index("Month")
)

#printing, but maybe remove it before committing?
display(long_df)
print("Rows:", len(long_df))

,APT_ICAO,delay_type
Month,,
2023-01,EDDF,Equipment (non-ATC)
2023-02,EDDF,Weather
2023-03,EDDF,Weather
2023-04,EDDF,Weather
2023-05,EDDF,Aerodrome Capacity
...,...,...
2024-08,LTFM,Weather
2024-09,LTFM,Weather
2024-10,LTFM,Weather


Rows: 480
